# Obsidian 助手 V2.1 验证 Notebook

目标: 对比 Baseline 与 (智能路由+模型适配器) 的性能与 Token 消耗差异，并预留缓存与压缩测试单元。

包含内容:
1. 环境与导入
2. 助手构建 (Baseline vs Enhanced)
3. 测试查询集定义
4. A/B 运行与度量
5. 结果汇总 (节省比例)
6. 占位: 缓存/压缩/语义索引后续补充

> 注: 运行前请确认已安装必要依赖 (LangChain、Qwen/DeepSeek SDK、Tavily 搜索等)。

## 1. 导入与辅助类定义

In [5]:
# 环境与路径配置 + 导入 (使用封装 inject_deepagents_paths)
import os, sys, time, json, importlib
from pathlib import Path
from typing import List, Dict

# 路径与包环境统一注入
try:
    from path_utils import inject_deepagents_paths
    inject_deepagents_paths(verbose=False)
except Exception as e:
    print(f"⚠️ 路径注入失败 fallback: {e}")
    cur = Path.cwd()
    libs = cur / "deepagents_official" / "libs"
    if libs.exists() and str(libs) not in sys.path:
        sys.path.insert(0, str(libs))

# 环境变量加载
try:
    from dotenv import load_dotenv
    load_dotenv()
except Exception:
    pass

print("🔐 DASHSCOPE_API_KEY:", "已设置" if os.getenv("DASHSCOPE_API_KEY") else "未设置")
print("🔐 TAVILY_API_KEY:", "已设置" if os.getenv("TAVILY_API_KEY") else "未设置")

# 导入 deepagents
try:
    import deepagents
    from deepagents import create_deep_agent
    print("✅ deepagents 导入成功:", deepagents.__file__)
except Exception as e:
    print("❌ deepagents 导入失败:", e)

# 导入 obsidian_assistant
try:
    import obsidian_assistant
    from obsidian_assistant import create_obsidian_assistant_v2, get_model_adapter, SmartRouter
    print("✅ obsidian_assistant 包导入成功")
except Exception as e:
    print("❌ obsidian_assistant 导入失败:", e)
    create_obsidian_assistant_v2 = None

# TokenCounter
try:
    from obsidian_assistant.token_counter import TokenCounter
except Exception:
    class TokenCounter:
        def __init__(self):
            self.records = []
        def start_counting(self):
            pass
    print("⚠️ 使用占位 TokenCounter")

token_counter = TokenCounter()

class PerformanceMonitor:
    def __init__(self):
        self.metrics = {
            'total_queries': 0,
            'total_time': 0.0,
            'total_prompt_tokens': 0,
            'total_completion_tokens': 0,
            'total_cost': 0.0,
            'web_queries': 0,
            'local_queries': 0,
        }
        self.records = []

    def log(self, route: str, usage: dict, elapsed: float):
        m = self.metrics
        m['total_queries'] += 1
        m['total_time'] += elapsed
        pt = usage.get('prompt_tokens', 0)
        ct = usage.get('completion_tokens', 0)
        m['total_prompt_tokens'] += pt
        m['total_completion_tokens'] += ct
        m['total_cost'] += usage.get('cost', 0.0)
        if route == 'web_first':
            m['web_queries'] += 1
        else:
            m['local_queries'] += 1
        self.records.append({
            'route': route,
            'prompt_tokens': pt,
            'completion_tokens': ct,
            'total_tokens': usage.get('total_tokens', pt+ct),
            'cost': usage.get('cost', 0.0),
            'time': elapsed,
        })

    def report(self):
        m = self.metrics
        if m['total_queries'] == 0:
            return {}
        return {
            'avg_time': m['total_time']/m['total_queries'],
            'avg_prompt_tokens': m['total_prompt_tokens']/m['total_queries'],
            'avg_completion_tokens': m['total_completion_tokens']/m['total_queries'],
            'avg_total_tokens': (m['total_prompt_tokens']+m['total_completion_tokens'])/m['total_queries'],
            'avg_cost': m['total_cost']/m['total_queries'],
            'web_ratio': m['web_queries']/m['total_queries'],
        }

print('✅ 环境与依赖导入完成 (使用 path_utils)')

🔐 DASHSCOPE_API_KEY: 已设置
🔐 TAVILY_API_KEY: 已设置
✅ deepagents 导入成功: /Users/yf/Documents/GitHub/deepagents/deepagents_official/libs/deepagents/deepagents/__init__.py
✅ obsidian_assistant 包导入成功
⚠️ 使用占位 TokenCounter
✅ 环境与依赖导入完成 (使用 path_utils)


## 2. 构建助手实例 (Baseline vs Enhanced)

In [ ]:
OBSIDIAN_PATH = './vault'  # 替换为真实路径
MODEL_NAME = 'qwen-turbo'

# 可选：设置环境变量控制详细日志
os.environ['OBSIDIAN_ASSISTANT_VERBOSE'] = '0'  # '1' / 'true' 可开启创建阶段详细输出

# Baseline: 禁用适配器 & 路由 (静默)
baseline = create_obsidian_assistant_v2(
    docs_path=OBSIDIAN_PATH,
    model_name=MODEL_NAME,
    enable_model_adapter=False,
    enable_smart_routing=False,
    verbose=False,
 )

# Enhanced: 启用适配器 & 路由 (由 env 控制)
enhanced = create_obsidian_assistant_v2(
    docs_path=OBSIDIAN_PATH,
    model_name=MODEL_NAME,
    enable_model_adapter=True,
    enable_smart_routing=True,
    verbose=None,  # None -> 读取环境变量
 )

print('助手实例构建完成 -> baseline / enhanced')

## 3. 定义测试查询集

In [ ]:
test_queries = [
    '如何创建双向链接？',
    '最新的 Obsidian 插件推荐有哪些？',
    '请总结我笔记里关于任务管理的最佳实践',
    '2025 年 AI 笔记工作流趋势是什么？',
    '怎样在本地库中查找引用未使用的笔记？',
    '生成一个关于我 Obsidian 每日回顾的模板示例',
    '推荐现在最活跃的知识管理社区',
    '笔记链接结构优化的步骤',
    '今年有哪些新特性值得关注',
    '总结 markdown 中表格快速编辑技巧'
]
len(test_queries)

## 4. A/B 运行逻辑定义

In [ ]:
baseline_monitor = PerformanceMonitor()
enhanced_monitor = PerformanceMonitor()

baseline_results = []
enhanced_results = []

coverage_values_baseline = []
coverage_values_enhanced = []

source_counts_baseline = []
source_counts_enhanced = []

def run_structured(assistant, query: str):
    start = time.time()
    result = assistant.invoke({"messages": [("user", query)]})
    elapsed = time.time() - start
    answer = result.get('answer')
    route = result.get('route_strategy') or 'n/a'
    usage = result.get('token_usage', {})
    coverage = result.get('route_coverage')
    time_sensitive = result.get('time_sensitive')
    sources = result.get('sources', [])
    return {
        'query': query,
        'answer': answer,
        'route': route,
        'elapsed': elapsed,
        'usage': usage,
        'coverage': coverage,
        'time_sensitive': time_sensitive,
        'sources': sources,
        'total_tokens': usage.get('total_tokens'),
        'cost': usage.get('cost'),
    }

for q in test_queries:
    b = run_structured(baseline, q)
    baseline_monitor.log(b['route'], b['usage'], b['elapsed'])
    baseline_results.append(b)
    if b['coverage'] is not None:
        coverage_values_baseline.append(b['coverage'])
    source_counts_baseline.append(len(b['sources']))

    e = run_structured(enhanced, q)
    enhanced_monitor.log(e['route'], e['usage'], e['elapsed'])
    enhanced_results.append(e)
    if e['coverage'] is not None:
        coverage_values_enhanced.append(e['coverage'])
    source_counts_enhanced.append(len(e['sources']))

print('测试完成: 共运行', len(test_queries), '条查询')

## 5. 汇总对比结果

In [ ]:
baseline_report = baseline_monitor.report()
enhanced_report = enhanced_monitor.report()

import json, statistics

avg_coverage_baseline = statistics.mean(coverage_values_baseline) if coverage_values_baseline else None
avg_coverage_enhanced = statistics.mean(coverage_values_enhanced) if coverage_values_enhanced else None
avg_sources_baseline = statistics.mean(source_counts_baseline) if source_counts_baseline else 0
avg_sources_enhanced = statistics.mean(source_counts_enhanced) if source_counts_enhanced else 0

comparison = {
    'baseline': baseline_report,
    'enhanced': enhanced_report,
    'tokens_reduction_pct': (
        f"{(baseline_report.get('avg_total_tokens',0)-enhanced_report.get('avg_total_tokens',0))/baseline_report.get('avg_total_tokens',1)*100:.1f}%"
        if baseline_report.get('avg_total_tokens') else 'n/a'
    ),
    'cost_reduction_pct': (
        f"{(baseline_report.get('avg_cost',0)-enhanced_report.get('avg_cost',0))/baseline_report.get('avg_cost',1)*100:.1f}%"
        if baseline_report.get('avg_cost') else 'n/a'
    ),
    'web_ratio_change_pct': (
        f"{(baseline_report.get('web_ratio',0)-enhanced_report.get('web_ratio',0))/baseline_report.get('web_ratio',1)*100:.1f}%"
        if baseline_report.get('web_ratio') else 'n/a'
    ),
    'avg_coverage_baseline': avg_coverage_baseline,
    'avg_coverage_enhanced': avg_coverage_enhanced,
    'coverage_change_pct': (
        f"{((avg_coverage_baseline or 0)-(avg_coverage_enhanced or 0))/(avg_coverage_baseline or 1)*100:.1f}%" if avg_coverage_baseline else 'n/a'
    ),
    'avg_sources_baseline': avg_sources_baseline,
    'avg_sources_enhanced': avg_sources_enhanced,
    'sources_change_pct': (
        f"{(avg_sources_baseline-avg_sources_enhanced)/avg_sources_baseline*100:.1f}%" if avg_sources_baseline else 'n/a'
    )
}
json.dumps(comparison, ensure_ascii=False, indent=2)

## 6. 缓存 / 压缩 / 语义索引后续扩展占位

后续功能上线后在此补充测试单元: 
- 缓存命中率对比 (首次 vs 重复查询)
- 网页结果压缩前后 token 对比
- 语义缓存相似度触发测试

可新增单元: `run_cache_trials()` 与 `measure_compression()`。

## 7. 初步结论与改进建议 (运行后填写)

- Tokens 平均降低: (待运行)
- 成本平均降低: (待运行)
- Web 搜索比例变化: (待运行)

下一步: 
1. 接入缓存层并在本 Notebook 增加缓存对比单元。
2. 接入结果压缩器并记录压缩率。
3. 引入真实 token 统计替换 rough 估算。